## About the data:
polyps are precursors to colorectal cancer, and is found in nearly half of the individuals at age 50 having a screening colonoscopy, and are increasing with age. Colonoscopy is the gold standard for detection and assessment of these polyps with subsequent biopsy and removal of the polyps. Early disease detection has a huge impact on survival from colorectal cancer, and polyp detection is therefore important. In addition, several studies have shown that polyps are often overlooked during colonoscopies, with polyp miss rates of 14%-30% depending on the type and size of the polyps. Increasing the detection of polyps has been shown to decrease risk of colorectal cancer. Thus, automatic detection of more polyps at an early stage can play a crucial role in improving both prevention of and survival from colorectal cancer. This is the main motivation behind the development of a Kvasir-SEG dataset.

The Kvasir-SEG dataset contains 1000 polyp images and their corresponding ground truth from the Kvasir Dataset v2. The resolution of the images contained in Kvasir-SEG varies from 332x487 to 1920x1072 pixels.

# Importing Libraries

In [ ]:
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import plotly.express as px
import plotly.io as pio
from plotly.offline import iplot
from sklearn.metrics import confusion_matrix, classification_report,roc_curve, auc

pio.templates["plotly_dark"].layout.colorway = px.colors.qualitative.Set2
pio.templates.default = "plotly_dark" 

# Loading images and masks

In [ ]:
images_dir = r"kvasir-seg/Kvasir-SEG/images"
masks_dir = r"kvasir-seg/Kvasir-SEG/masks"

In [ ]:
def load_image_and_mask(image_path,mask_path):
    image_size = (256,256)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,image_size)
    
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask,image_size)
    mask = np.expand_dims(mask,axis=-1)
    mask = np.where(mask>0,1,0).astype(np.float32)
    
    return img,mask

In [ ]:
def load_dataset(image_paths,mask_paths):
    imgs = []
    masks = []
    for i,m in zip(image_paths,mask_paths):
        img,mask  = load_image_and_mask(i,m)
        imgs.append(img)
        masks.append(mask)
    
    return np.array(imgs),np.array(masks)

In [ ]:
img_p = r"kvasir-seg/Kvasir-SEG/images/cju0roawvklrq0799vmjorwfv.jpg"
msk_p = r"kvasir-seg/Kvasir-SEG/masks/cju0roawvklrq0799vmjorwfv.jpg"
img = cv2.imread(img_p)
msk = cv2.imread(msk_p,cv2.IMREAD_GRAYSCALE)
plt.figure(figsize= (12,15))
plt.subplot(1,3,1)
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.subplot(1,3,2)
plt.imshow(cv2.cvtColor(msk, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(1,3,3)
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.imshow(cv2.cvtColor(msk, cv2.COLOR_BGR2RGB),alpha = 0.5)
plt.axis("off")

plt.show()

In [ ]:
images_paths = [os.path.join(images_dir,f) for f in os.listdir(images_dir)]
masks_paths = [os.path.join(masks_dir,f) for f in os.listdir(masks_dir)]

In [ ]:
len(images_paths)

In [ ]:
len(masks_paths)

In [ ]:
images,masks = load_dataset(images_paths,masks_paths)

# Creating U-Net model

In [ ]:
def u_net():
    inputs = layers.Input(shape = (256,256,3))
    inputs = layers.Rescaling(scale= 1./255)(inputs)
    
    #Encoder
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(inputs)
    c1 = layers.BatchNormalization()(c1)
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(p1)
    c2 = layers.BatchNormalization()(c2)
    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(p2)
    c3 = layers.BatchNormalization()(c3)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(p3)
    c4 = layers.BatchNormalization()(c4)
    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    c5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(p4)
    c5 = layers.BatchNormalization()(c5)
    c5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(c5)
    p5 = layers.MaxPooling2D((2, 2))(c5)
    
    c6 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(p5)
    c6 = layers.BatchNormalization()(c6)
    c6 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(c6)


    # Decoder
    u7 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c5])
    c7 = layers.Conv2D(512, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(u7)
    c7 = layers.BatchNormalization()(c7)
    c7 = layers.Conv2D(512, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(c7)

    
    u8 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c4])
    c8 = layers.Conv2D(256, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(u8)
    c8 = layers.BatchNormalization()(c8)
    c8 = layers.Conv2D(256, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(c8)


    u9 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same',kernel_initializer = "he_normal")(c8)
    u9 = layers.concatenate([u9, c3])
    c9 = layers.Conv2D(128, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(u9)
    c9 = layers.BatchNormalization()(c9)
    c9 = layers.Conv2D(128, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(c9)


    u10 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same',kernel_initializer = "he_normal")(c9)
    u10 = layers.concatenate([u10, c2])
    c10 = layers.Conv2D(64, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(u10)
    c10 = layers.BatchNormalization()(c10)
    c10 = layers.Conv2D(64, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(c10)

    
    u11 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c10)
    u11 = layers.concatenate([u11, c1])
    c11 = layers.Conv2D(32, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(u11)
    c11 = layers.BatchNormalization()(c11)
    c11 = layers.Conv2D(16, (3, 3), activation='relu', padding='same',kernel_initializer = "he_normal")(c11)

    
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c11)
    
    model = models.Model(inputs=[inputs],outputs = [outputs])
    model.compile(optimizer = 'adam', loss = "binary_crossentropy",metrics = ["accuracy",tf.metrics.BinaryIoU(target_class_ids=(0, 1))])
    
    return model

In [ ]:
model = u_net()

In [ ]:
model.summary()

# Training Model

In [ ]:
check_point = ModelCheckpoint('U-Net.keras', monitor='val_accuracy', save_best_only=True, mode='max')
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  
    factor=0.7,          
    patience=5,          
    min_lr=1e-7,        
    verbose=1       
)

In [ ]:
history = model.fit(x=images,y=masks, epochs=50, validation_split = 0.1, batch_size = 16 ,callbacks = [check_point,reduce_lr])

# Evaluating model

In [ ]:
vals = pd.DataFrame(history.history)
vals = vals.rename(columns={"loss":"Train Loss","val_loss":"Validation Loss",\
                            "accuracy":"Train Accuracy","val_accuracy":"Validation Accuracy",
                             "binary_io_u":"Train IoU","val_binary_io_u":"Validation IoU"
                           })

In [ ]:
vals

In [ ]:
iplot(px.line(
    data_frame = vals,
    y = ["Train Accuracy","Validation Accuracy"],
    labels = {"index":"epochs","value":"accuracy"},
    title = "Accuracy"
))

In [ ]:
iplot(px.line(
    data_frame = vals,
    y = ["Train Loss","Validation Loss"],
    labels = {"index":"epochs","value":"Loss"},
    title = "Loss"
))

In [ ]:
iplot(px.line(
    data_frame = vals,
    y = ["Train IoU","Validation IoU"],
    labels = {"index":"epochs","value":"IoU"},
    title = "IoU(Intersection over Union)"
))

# Predicting an image from segmentation dataset

In [ ]:
img = cv2.imread(r"kvasir-seg/Kvasir-SEG/images/cju0u2g7pmnux0801vkk47ivj.jpg")
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = cv2.resize(img,(256,256))
img = np.array(img)
msk = cv2.imread("kvasir-seg/Kvasir-SEG/masks/cju0u2g7pmnux0801vkk47ivj.jpg",\
                 cv2.IMREAD_GRAYSCALE)
msk = cv2.resize(msk,(256,256))
predictions = model.predict(np.expand_dims(img, axis=0), verbose=0)
predictions = tf.where(predictions > 0.5, 1.0, 0.0)


plt.figure(figsize=(22, 15))
plt.subplot(2, 3, 1)
plt.title("original image")
plt.imshow(img)
plt.axis("off")

plt.subplot(2, 3, 2)
plt.title("original mask")
plt.imshow(msk)
plt.axis("off")

plt.subplot(2, 3, 3)
plt.title("original image with original mask")
plt.imshow(img)
plt.imshow(np.squeeze(msk),alpha=0.5)
plt.axis("off")

plt.subplot(2,3,4)
plt.title("original image")
plt.imshow(img)
plt.axis("off")

plt.subplot(2,3,5)
plt.title("predicted mask")
plt.imshow(np.squeeze(predictions))
plt.axis("off")

plt.subplot(2, 3, 6)
plt.title("original image with predicted mask")
plt.imshow(img)
plt.imshow(np.squeeze(predictions),alpha=0.5)
plt.axis("off")

# Predicting an image from the classification dataset

In [ ]:
img = cv2.imread(r"test img.jpg")
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = cv2.resize(img,(256,256))
predictions = model.predict(np.expand_dims(img, axis=0), verbose=0)
predictions = tf.where(predictions > 0.5, 1.0, 0.0)


plt.figure(figsize=(22, 15))
plt.subplot(1, 3, 1)
plt.title("original image")
plt.imshow(img)
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("predicted  mask")
plt.imshow(np.squeeze(predictions))
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("original image with predicted mask")
plt.imshow(img)
plt.imshow(np.squeeze(predictions),alpha=0.5)
plt.axis("off")